In [12]:
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, SelectFwe, f_classif, mutual_info_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np

In [3]:
X_train = np.array(pd.read_csv('train.csv', header=None))
y_train = np.array(pd.read_csv('trainlabels.csv', header=None)).reshape(51147)
X_test = np.array(pd.read_csv('test.csv', header=None))

In [46]:
train_inds, val_inds = next(StratifiedKFold(n_splits=10, shuffle=True).split(X_train, y_train))

X_val, y_val = X_train[val_inds], y_train[val_inds]
X_train, y_train = X_train[train_inds], y_train[train_inds]

In [7]:
X_train.shape

(46032, 117)

In [47]:
minfos = mutual_info_classif(X_train, y_train)

In [48]:
minds = infos.argsort()[-20:]
X2 = X_train[:, minds]
X2.shape

(37284, 20)

In [45]:
m1 = minds.copy()

X: uncorrelated X_train: 0.74
X2: top 50 mi values of X: 0.81
X3: top 30 mi values of X: 0.87
X4: top 20 mi values of X: 0.80
X3: mi values of X_train

In [49]:
clf = SVC(probability=True)
clf.fit(X2, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [50]:
probs = clf.predict_log_proba(X_val[:,minds])[:,1]
roc_auc_score(y_val, probs)

0.79740238679669473

In [92]:
probs = pd.DataFrame(probs)
probs.rename(index='Id')
probs.index -= 1
probs.columns = ['Prediction']
probs.to_csv('out2.csv')
probs

TypeError: 'str' object is not callable